# Simple Demo
This notebook will run through 12 sources and 7 destinations.  
It will create a 12x7 walking matrix of amenities in Hyde Park, Chicago.  
Then, it will run through a basic version of each access and coverage metric to illustrate it's usage.

In [2]:
# Check to see what version of spatial access the user is using
! pip3 show spatial-access

Name: spatial-access
Version: 1.0.0
Summary: UNKNOWN
Home-page: https://github.com/GeoDaCenter/spatial_access
Author: Logan Noel (lmnoel)
Author-email: lnoel@uchicago.edu
License: GPL
Location: /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages
Requires: fiona, cython, matplotlib, jellyfish, geopandas, psutil, pandas, numpy, osmnet, scipy, geopy, shapely, tables, scikit-learn, atlas, descartes, rtree, rtree
Required-by: 
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
cd ../..

/Users/irenefarah/Documents/GitHub/spatial_access


## Creating the Matrix

In [4]:
from spatial_access.p2p import *

SourceNotBuiltException: 

In [4]:
import pandas as pd
sources_df = pd.read_csv('./data/input_data/sources/hyde_park_tracts.csv')
dests_df = pd.read_csv('./data/input_data/destinations/hyde_park_dests.csv')


In [5]:
sources_df

,geoid10,lon,lat,Pop2014,Pov14,community
0,17031836300,-87.601757,41.801532,6465,234,41
1,17031836200,-87.601284,41.790469,1329,47,41
2,17031410100,-87.579323,41.801497,1956,551,41
3,17031410200,-87.594269,41.801668,1248,362,41
4,17031410500,-87.603745,41.797827,2630,717,41
5,17031410600,-87.598946,41.797971,2365,703,41
6,17031411100,-87.589702,41.790449,2246,154,41
7,17031410700,-87.594198,41.798040,1959,453,41
8,17031410800,-87.589626,41.797960,3201,741,41
9,17031410900,-87.576659,41.797874,2923,607,41


In [6]:
dests_df

,name,lon,lat,category,target
0,Museum of Science and Industry,-87.583131,41.790883,Museum,400
1,Medici,-87.593738,41.791438,Restaurant,50
2,Valois,-87.588328,41.799663,Restaurant,30
3,DuSable Museum,-87.607132,41.791985,Museum,100
4,Whole Foods,-87.587949,41.801978,Supermarket,50
5,Hyde Park Produce,-87.595524,41.799942,Supermarket,35
6,Jewel Osco,-87.607225,41.784580,Supermarket,70


In [7]:
#assymetrix matrix, different source and dest files
matrix = TransitMatrix(network_type='walk',
                       primary_hints={'idx' : 'geoid10', 'population': 'skip', 'lat': 'lat', 'lon': 'lon'},
                       secondary_hints={'idx': 'name', 'capacity': 'skip', 'category': 'category', 'lat': 'lat', 'lon': 'lon'},
                       primary_input='./data/input_data/sources/hyde_park_tracts.csv',
                       secondary_input='./data/input_data/destinations/hyde_park_dests.csv')


In [8]:
matrix.process()

INFO:spatial_access.p2p:Approx area of bounding box: 141.64 sq. km


Requesting network data within bounding box from Overpass API in 1 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](41.73458000,-87.65722500,41.85197800,-87.52665910);>;);out;'}"
Downloaded 6,962.4KB from www.overpass-api.de in 9.37 seconds
Downloaded OSM network data within bounding box from Overpass API in 1 request(s) and 9.50 seconds
Returning OSM data with 39,275 nodes and 10,896 ways...
Edge node pairs completed. Took 17.76 seconds
Returning processed graph with 18,265 nodes and 27,897 edges...
Completed OSM data download and Pandana node and edge table creation in 28.49 seconds


//anaconda/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['hgv', 'highway', 'maxspeed', 'name', 'oneway', 'ref']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
INFO:spatial_access.p2p:Finished querying osm
INFO:spatial_access.p2p:All operations completed in 28.99 seconds


In [9]:
matrix.write_csv('./data/output_data/matrices/simple_demo_matrix.csv')

INFO:spatial_access.p2p:Wrote to data/output_data/matrices/simple_demo_matrix.csv in 0.00 seconds


## Access Metrics (Origin Attributes)

In [10]:
from spatial_access.Models import *

### Access Model

The first line of code is defining Access Model using the previously generated matrix above with a linear decay function. If the user specifies **`transit_matrix_filename=None`**, the matrix will be estimated 'on the fly'.  
The second line of code is defining the density and variety weights.

In [11]:
access = AccessModel(network_type='walk',
                     transit_matrix_filename='./data/output_data/matrices/simple_demo_matrix.csv',
                     sources_filename='./data/input_data/sources/hyde_park_tracts.csv',
                     destinations_filename='./data/input_data/destinations/hyde_park_dests.csv',
                     source_column_names={'idx' : 'geoid10', 'population': 'skip', 'lat': 'lat', 'lon': 'lon'},
                     dest_column_names={'idx': 'name', 'capacity': 'skip', 'category': 'category', 'lat': 'lat', 'lon': 'lon'},
                     decay_function = 'linear')

In [12]:
category_dict = {
    "Museum": [5, 5, 3],
    "Restaurant": [10, 10],
    "Supermarket": [10, 7, 5]
}

In [13]:
access.calculate(upper_threshold=1800,
                 normalize=False,
                 category_weight_dict=category_dict)

INFO:spatial_access.BaseModel:Using weights: {'Museum': [5, 5, 3], 'Supermarket': [10, 7, 5], 'Restaurant': [10, 10]}


TypeError: from_dict() got an unexpected keyword argument 'columns'

In [17]:
access.model_results.head()

,all_categories_score,Museum_score,Supermarket_score,Restaurant_score
17031836300,19.318333,2.108333,9.826667,7.383333
17031836200,22.553333,4.983333,8.342222,9.227778
17031410100,18.303889,2.011111,8.398333,7.894444
17031410200,25.826667,1.647222,12.301667,11.877778
17031410500,21.282778,3.519444,9.713333,8.050000


In [18]:
access.model_results.to_csv('./data/output_data/models/simple_demo_accessMod.csv')

### Access Time: Time to closest destination

Here, we are defining the Access Time model using the sources and destinations csv. 

In [14]:
accessT = AccessTime(network_type='walk',
                     transit_matrix_filename='./data/output_data/matrices/simple_demo_matrix.csv',
                     sources_filename='./data/input_data/sources/hyde_park_tracts.csv',
                     destinations_filename='./data/input_data/destinations/hyde_park_dests.csv',
                     source_column_names={'idx' : 'geoid10', 'population': 'skip', 'lat': 'lat', 'lon': 'lon'},
                     dest_column_names={'idx': 'name', 'capacity': 'skip', 'category': 'category', 'lat': 'lat', 'lon': 'lon'}
                    )

In [15]:
accessT.calculate()

IndexError: unordered_map::at: key not found

In [ ]:
accessT.model_results.head()

In [21]:
accessT.model_results.to_csv('data/output_data/models/simple_demo_accessT.csv')

### Access Count: Number of Destinations in Range

Access Count measures the number of destinations within a catchment area. In this case, the catchment area is 1800 seconds (30 minutes) of walking.

In [16]:
accessC = AccessCount(network_type='walk',
                     transit_matrix_filename='./data/output_data/matrices/simple_demo_matrix.csv',
                     sources_filename='./data/input_data/sources/hyde_park_tracts.csv',
                     destinations_filename='./data/input_data/destinations/hyde_park_dests.csv',
                     source_column_names={'idx' : 'geoid10', 'population': 'skip', 'lat': 'lat', 'lon': 'lon'},
                     dest_column_names={'idx': 'name', 'capacity': 'skip', 'category': 'category', 'lat': 'lat', 'lon': 'lon'}
                     )

In [17]:
accessC.calculate(upper_threshold=1800)

IndexError: unordered_map::at: key not found

In [34]:
accessC.model_results.head()

,count_in_range_Museum,count_in_range_Supermarket,count_in_range_Restaurant,count_in_range_all_categories
17031836300,1,3,2,6
17031836200,2,3,2,7
17031410100,1,2,2,5
17031410200,2,2,2,6
17031410500,2,3,2,7


In [25]:
accessC.model_results.to_csv('data/output_data/models/simple_demo_accessC.csv')

### Access Sum: sum of an attribute of facility within a catchment area

In [18]:
accessS = AccessSum(network_type='walk',
                     transit_matrix_filename='data/output_data/matrices/simple_demo_matrix.csv',
                     sources_filename='data/input_data/sources/hyde_park_tracts.csv',
                     destinations_filename='data/input_data/destinations/hyde_park_dests.csv',
                     source_column_names={'idx' : 'geoid10', 'population': 'skip', 'lat': 'lat', 'lon': 'lon'},
                     dest_column_names={'idx': 'name', 'capacity': 'target', 'category': 'category', 'lat': 'lat', 'lon': 'lon'}
                   )

In [19]:
accessS.calculate(upper_threshold=1800)

TypeError: from_dict() got an unexpected keyword argument 'columns'

In [42]:
accessS.model_results.head()

,sum_in_range_Museum,sum_in_range_Supermarket,sum_in_range_Restaurant,sum_in_range_all_categories
17031836300,100,155,80,335
17031836200,500,155,80,735
17031410100,400,85,80,565
17031410200,500,85,80,665
17031410500,500,155,80,735


In [40]:
accessS.model_results.to_csv('./data/output_data/simple_demo_accessS.csv')

### Destination Sum: capacity and capacity per capita of providers in area

In [94]:
d_sum = DestSum(network_type='walk',
                sources_filename='d./ata/input_data/sources/hyde_park_tracts.csv',
                destinations_filename='./data/input_data/destinations/hyde_park_dests.csv',
                source_column_names={'idx' : 'geoid10', 'population': 'skip', 'lat': 'lat', 'lon': 'lon'},
                dest_column_names={'idx': 'name', 'capacity': 'target', 'category': 'category', 'lat': 'lat', 'lon': 'lon'}
               )

In [95]:
d_sum.calculate()

,Museum,Supermarket,Restaurant,all_categories,Museum_per_capita,Supermarket_per_capita,Restaurant_per_capita,all_categories_per_capita
spatial_index,,,,,,,,
HYDE PARK,400.0,85.0,80.0,565.0,44.444444,9.444444,8.888889,62.777778
WASHINGTON PARK,100.0,0.0,0.0,100.0,NaN,NaN,NaN,NaN
WOODLAWN,0.0,70.0,0.0,70.0,NaN,NaN,NaN,NaN


In [45]:
d_sum.aggregated_results.head()

,Museum,Supermarket,Restaurant,all_categories,Museum_per_capita,Supermarket_per_capita,Restaurant_per_capita,all_categories_per_capita
spatial_index,,,,,,,,
HYDE PARK,400.0,85.0,80.0,565.0,0.017291,0.003674,0.003458,0.024423
WASHINGTON PARK,100.0,0.0,0.0,100.0,NaN,NaN,NaN,NaN
WOODLAWN,0.0,70.0,0.0,70.0,NaN,NaN,NaN,NaN


In [37]:
d_sum.aggregated_results.to_csv('./data/output_data/simple_demo_destsum.csv')

## Coverage Metrics (Destinations)

### Coverage Score

In [46]:
cov = Coverage(network_type='walk',
               transit_matrix_filename='./data/output_data/matrices/simple_demo_matrix.csv',
               sources_filename='./data/input_data/sources/hyde_park_tracts.csv',
               destinations_filename='./data/input_data/destinations/hyde_park_dests.csv',
               source_column_names={'idx' : 'geoid10', 'population': 'Pop2014', 'lat': 'lat', 'lon': 'lon'},
               dest_column_names={'idx': 'name', 'capacity': 'target', 'category': 'category', 'lat': 'lat', 'lon': 'lon'}
              )

The variables in your data set are:
>  geoid10
>  lon
>  lat
>  Pop2014
>  Pov14
>  community


Enter the longitude coordinate:  lon
Enter the latitude coordinate:  lat
Enter the index name:  geoid10


The variables in your data set are:
>  name
>  lon
>  lat
>  category
>  target


Enter the longitude coordinate:  lon
Enter the latitude coordinate:  lat
Enter the index name:  name


INFO:spatial_access.p2p:Approx area of bounding box: 141.64 sq. km
INFO:spatial_access.p2p:All operations completed in 10.87 seconds


...

Enter the population variable:  Pop2014


The variables in your data set are:
>  name
>  lon
>  lat
>  category
>  target
If you have no capacity variable, write "skip" (no quotations)


Enter the capacity variable:  target


If you have no category variable, write "skip" (no quotations)


Enter the category variable:  category


In [47]:
cov.calculate(upper_threshold=1800)

,service_pop,percap_spending,category
Museum of Science and Industry,24861,0.016089,Museum
DuSable Museum,23134,0.004323,Museum
Whole Foods,31326,0.001596,Supermarket
Hyde Park Produce,31326,0.001117,Supermarket
Jewel Osco,16726,0.004185,Supermarket
Medici,31326,0.001596,Restaurant
Valois,31326,0.000958,Restaurant


In [16]:
cov.model_results.to_csv('./data/output_data/models/simple_demo_cov.csv')

### Two Stage Floating Catchment Area

In [48]:
tsfca = TSFCA(network_type='walk',
              transit_matrix_filename='./data/output_data/matrices/simple_demo_matrix.csv',
              sources_filename='./data/input_data/sources/hyde_park_tracts.csv',
              destinations_filename='./data/input_data/destinations/hyde_park_dests.csv',
              source_column_names={'idx' : 'geoid10', 'population': 'Pop2014', 'lat': 'lat', 'lon': 'lon'},
              dest_column_names={'idx': 'name', 'capacity': 'target', 'category': 'category', 'lat': 'lat', 'lon': 'lon'}
             )

The variables in your data set are:
>  geoid10
>  lon
>  lat
>  Pop2014
>  Pov14
>  community


Enter the longitude coordinate:  lon
Enter the latitude coordinate:  lat
Enter the index name:  geoid10


The variables in your data set are:
>  name
>  lon
>  lat
>  category
>  target


Enter the longitude coordinate:  lon
Enter the latitude coordinate:  lat
Enter the index name:  name


INFO:spatial_access.p2p:Approx area of bounding box: 141.64 sq. km
INFO:spatial_access.p2p:All operations completed in 8.62 seconds


...

Enter the population variable:  Pop2014


The variables in your data set are:
>  name
>  lon
>  lat
>  category
>  target
If you have no capacity variable, write "skip" (no quotations)


Enter the capacity variable:  target


If you have no category variable, write "skip" (no quotations)


Enter the category variable:  category


In [49]:
tsfca.calculate(upper_threshold=1800)

,percap_spend_Museum,percap_spend_Supermarket,percap_spend_Restaurant,percap_spend_all_categories
17031836300,0.004323,0.006899,0.002554,0.013775
17031836200,0.020412,0.006899,0.002554,0.029864
17031410100,0.016089,0.002713,0.002554,0.021357
17031410200,0.020412,0.002713,0.002554,0.025679
17031410500,0.020412,0.006899,0.002554,0.029864
17031410600,0.020412,0.006899,0.002554,0.029864
17031411100,0.020412,0.006899,0.002554,0.029864
17031410700,0.020412,0.002713,0.002554,0.025679
17031410800,0.020412,0.002713,0.002554,0.025679
17031410900,0.016089,0.002713,0.002554,0.021357


In [11]:
tsfca.model_results.to_csv('./data/output_data/models/simple_demo_tsfca.csv')